In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import torch
from torchinfo import summary
from sade.configs.ve import toy_config

config = toy_config.get_config()
config.model

act: memswish
blocks_down: !!python/tuple
- 1
- 2
- 1
blocks_up: !!python/tuple
- 1
- 1
conv_size: 3
dilation: 1
dropout: 0.0
ema_rate: 0.9999
embedding_type: fourier
fourier_scale: 2.0
init_scale: 0.0
jit: false
learnable_embedding: false
name: ncsnpp3d
nf: 8
norm_num_groups: 2
num_scales: 10
resblock_pp: true
resblock_type: biggan
scale_by_sigma: true
self_attention: false
sigma_max: 274.0
sigma_min: 0.03
time_embedding_sz: 32

In [3]:
from sade.models.learning.toynet import ToyNet
model = ToyNet(config)
N=3
H=W=D = 16
x = torch.zeros(N, 2, H,W,D)
s = torch.ones(N, 1)
model(x,s).shape

2023-09-14 14:48:51.679719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/torch/lib:/usr/local/lib/python3.10/dist-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-09-14 14:48:51.679737: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Embedding shape: torch.Size([3, 32])


torch.Size([3, 2, 16, 16, 16])

In [7]:
summary(model, input_data=(x,s), depth=1)

Embedding shape: torch.Size([3, 32])


Layer (type:depth-idx)                   Output Shape              Param #
ToyNet                                   [3, 2, 16, 16, 16]        --
├─Conv3d: 1-1                            [3, 8, 16, 16, 16]        24
├─GaussianFourierProjection: 1-2         [3, 32]                   (16)
├─ResnetBlockBigGANpp: 1-3               [3, 8, 16, 16, 16]        4,032
├─ResnetBlockBigGANpp: 1-4               [3, 8, 8, 8, 8]           5,768
├─Upsample: 1-5                          [3, 8, 16, 16, 16]        --
├─ResnetBlockBigGANpp: 1-6               [3, 8, 16, 16, 16]        4,032
├─Conv3d: 1-7                            [3, 2, 16, 16, 16]        18
Total params: 13,890
Trainable params: 13,874
Non-trainable params: 16
Total mult-adds (M): 93.85
Input size (MB): 0.10
Forward/backward pass size (MB): 7.77
Params size (MB): 0.06
Estimated Total Size (MB): 7.92

In [5]:
upl = torch.nn.Upsample(scale_factor=2)

In [6]:
upl(x).shape

torch.Size([3, 2, 32, 32, 32])